In [1]:
import pandas as pd
import numpy as np
from m4_feats_functions import *

In [2]:
INPUT_DIR = 'kaggle/input/linking-writing-processes-to-writing-quality'
FEAT_STORE_DIR = 'feature_store'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

In [14]:
def agg_fe_df(logs):
    df = train_logs.groupby("id")[['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']].agg(
        ['mean', 'std', 'min', 'max', 'last', 'first', 'sem', 'median', 'sum']) # what about quantiles, IQR, etc.
    
    df.columns = ['_'.join(x) for x in df.columns]
    df = df.add_prefix("tmp_")
    df.reset_index(inplace=True)

    return df

agg_fe_df(train_logs)

,id,tmp_down_time_mean,tmp_down_time_std,tmp_down_time_min,tmp_down_time_max,tmp_down_time_last,tmp_down_time_first,tmp_down_time_sem,tmp_down_time_median,tmp_down_time_sum,...,tmp_cursor_position_sum,tmp_word_count_mean,tmp_word_count_std,tmp_word_count_min,tmp_word_count_max,tmp_word_count_last,tmp_word_count_first,tmp_word_count_sem,tmp_word_count_median,tmp_word_count_sum
0,001519c8,8.436548e+05,395112.665961,0,1797351,1797351,0,7813.679400,887190.0,2157225252,...,1818445,128.116152,76.498372,0,256,255,0,1.512819,132.0,327593
1,0022f953,4.882323e+05,384959.404177,0,1758219,1758219,0,7771.013336,377050.0,1198122181,...,1904809,182.714751,97.763090,0,323,320,0,1.973502,186.0,448382
2,0042269b,8.240508e+05,489500.796565,0,1766778,1766778,0,7611.375322,755141.0,3408274006,...,3025946,194.772727,108.935068,0,404,404,0,1.693860,193.0,805580
3,0059420b,7.440880e+05,385205.014399,0,1362999,1362999,0,9765.334758,806845.5,1157800969,...,844188,103.618895,61.882250,0,206,206,0,1.568777,108.5,161231
4,0075873a,6.348842e+05,405576.409034,0,1583920,1583920,0,8061.699636,608118.0,1606891904,...,1518729,125.082971,77.255054,0,252,252,0,1.535610,113.0,316585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,7.136349e+05,503882.020411,0,1769114,1769114,0,7319.568976,712628.0,3381915633,...,3667989,256.353661,118.093794,0,461,273,0,1.715472,297.0,1214860
2467,ffbef7e5,8.202342e+05,512744.745940,0,1777392,1777392,0,10048.025509,726672.5,2135889912,...,2661493,223.013057,126.627934,0,438,438,0,2.481470,227.5,580726
2468,ffccd6fd,1.205533e+06,514320.848199,0,1935791,1935791,0,9293.100430,1483043.0,3692547571,...,4009729,157.589292,61.236111,0,201,201,0,1.106456,201.0,482696
2469,ffec5b38,5.566335e+05,334477.976640,0,1488450,1488450,0,5874.366278,554027.0,1804605942,...,3866542,205.917027,118.473905,0,413,413,0,2.080732,205.0,667583


In [17]:
train_ids = train_logs.id
test_ids = test_logs.id

logs = pd.concat([train_logs, test_logs], axis=0)
logs = normalise_up_down_times(logs)

preprocessor = Preprocessor(seed=42)
feats = preprocessor.make_feats(logs)
nan_cols = feats.columns[feats[feats['id'].isin(train_ids)].isna().any()].tolist()
feats = feats.drop(columns=nan_cols)

essays = getEssays(logs)
sent_feats = compute_sentence_aggregations(essays)
par_feats = compute_paragraph_aggregations(essays)
word_feats = create_word_length_features(essays, 'essay', 'id', 'essay_words')
vector_feats = countvectorize_one_one(logs)

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 33/33 [01:13<00:00,  2.23s/it, column=word_count_change100, method=kurt]         


Engineering activity counts data


100%|██████████| 2474/2474 [00:00<00:00, 14767.92it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering event counts data


...

100%|██████████| 2474/2474 [00:00<00:00, 14329.16it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering punctuation counts data


100%|██████████| 2474/2474 [00:00<00:00, 14084.34it/s]


Engineering input words data
Engineering ratios data


/tmp/ipykernel_1276244/657098511.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
/tmp/ipykernel_1276244/657098511.py:217: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
/tmp/ipykernel_1276244/657098511.py:218: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all c

In [16]:
def countvectorize_one_one(logs):

    essays = getEssays(logs)
    c_vect = CountVectorizer(ngram_range=(1, 1))
    toks = c_vect.fit_transform(essays['essay']).todense()
    toks_df = pd.DataFrame(columns = [f'tok_{i}' for i in range(toks.shape[1])], data=toks)
    toks_df.reset_index(drop=True, inplace=True)

    return toks_df

# train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_1.pkl')
# test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_1.pkl')
countvectorize_one_one(train_logs)

100%|██████████| 2471/2471 [00:04<00:00, 590.46it/s]


,tok_0,tok_1,tok_2,tok_3,tok_4,tok_5,tok_6,tok_7,tok_8,tok_9,...,tok_17,tok_18,tok_19,tok_20,tok_21,tok_22,tok_23,tok_24,tok_25,tok_26
0,53,46,36,37,21,19,13,19,8,4,...,0,0,0,0,0,0,0,0,0,0
1,61,88,67,42,23,12,5,1,4,3,...,0,0,0,0,0,0,0,0,0,0
2,64,70,61,49,29,37,31,32,6,20,...,0,0,0,0,0,0,0,0,0,0
3,44,31,43,19,22,23,2,2,3,1,...,0,0,0,0,0,0,0,0,0,0
4,49,61,53,29,13,15,8,4,6,9,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,49,39,55,37,19,26,13,11,3,4,...,0,0,0,0,0,0,0,0,0,0
2467,74,78,102,51,34,30,23,8,3,0,...,0,0,0,0,0,0,0,0,0,0
2468,42,44,26,25,24,11,7,10,2,5,...,0,0,0,0,0,0,0,0,0,0
2469,80,72,66,53,37,40,13,13,12,15,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def countvectorize_one_one(train_logs, test_logs, train_feats, test_feats):

    tr_ids = train_feats.id
    tst_ids = test_feats.id
    tr_ts_logs = pd.concat([train_logs, test_logs], axis=0)
    tr_ts_feats = pd.concat([train_feats['id'], test_feats['id']], axis=0).reset_index(drop=True)

    essays = getEssays(tr_ts_logs)
    c_vect = CountVectorizer(ngram_range=(1, 1))
    toks = c_vect.fit_transform(essays['essay']).todense()
    toks_df = pd.DataFrame(columns = [f'tok_{i}' for i in range(toks.shape[1])], data=toks)
    toks_df.reset_index(drop=True, inplace=True)
    print(toks_df.shape, tr_ts_feats.shape)

    tr_ts_feats = pd.concat([tr_ts_feats, toks_df], axis=1)

    train_feats = tr_ts_feats[tr_ts_feats['id'].isin(tr_ids)]
    test_feats = tr_ts_feats[tr_ts_feats['id'].isin(tst_ids)]

    return train_feats, test_feats

train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_1.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_1.pkl')
train_, test_ = countvectorize_one_one(train_logs, test_logs, train_feats, test_feats)

100%|██████████| 2474/2474 [00:04<00:00, 572.73it/s]


(2474, 27) (2474,)


In [5]:
test_

,id,event_id_max,up_time_max,action_time_max,action_time_min,action_time_mean,action_time_std,action_time_quantile,action_time_sem,action_time_sum,...,tok_17,tok_18,tok_19,tok_20,tok_21,tok_22,tok_23,tok_24,tok_25,tok_26


In [8]:
file_name = 'count_vectorized_bigrams'

train_.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')
test_.to_pickle(f'{FEAT_STORE_DIR}/test_{file_name}.pkl')

In [3]:
preprocessor = Preprocessor(seed=42)
train_feats = preprocessor.make_feats(train_logs)
test_feats = preprocessor.make_feats(test_logs)
nan_cols = train_feats.columns[train_feats.isna().any()].tolist()
train_feats = train_feats.drop(columns=nan_cols)
test_feats = test_feats.drop(columns=nan_cols)

train_, test_ = process_action_time_activity(train_logs, test_logs)
train_feats = train_feats.merge(train_, on='id', how='left')
test_feats = test_feats.merge(test_, on='id', how='left')

train_essay = getEssays(train_logs)
test_essay = getEssays(test_logs)
train_ = create_word_length_features(train_essay, 'essay_words', 'id', 'words')
test_ = create_word_length_features(test_essay, 'essay_words', 'id', 'words')
train_feats = train_feats.merge(train_, on='id', how='left')
test_feats = test_feats.merge(test_, on='id', how='left')

train_sent_df = split_essays_into_sentences(train_essay)
train_ = compute_sentence_aggregations(train_sent_df)
test_ = compute_sentence_aggregations(split_essays_into_sentences(test_essay))
train_feats = train_feats.merge(train_, on='id', how='left')
test_feats = test_feats.merge(test_, on='id', how='left')

train_paragraph_df = split_essays_into_paragraphs(train_essay)
train_ = compute_paragraph_aggregations(train_paragraph_df)
test_ = compute_paragraph_aggregations(split_essays_into_paragraphs(test_essay))
train_feats = train_feats.merge(train_, on='id', how='left')
test_feats = test_feats.merge(test_, on='id', how='left')
##### feat_1
train_, test_ = process_feats_time_gap_activity(train_logs, test_logs)
train_feats = train_feats.merge(train_, on='id', how='left')
test_feats = test_feats.merge(test_, on='id', how='left')

train_, test_ = process_feats_action_time_gap(train_logs, test_logs)
train_feats = train_feats.merge(train_, on='id', how='left')
test_feats = test_feats.merge(test_, on='id', how='left')
##### feat_2
train_feats.shape

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 33/33 [01:29<00:00,  2.72s/it, column=word_count_change100, method=kurt]         


Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 12514.69it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering event counts data


...

Engineering punctuation counts data


100%|██████████| 3/3 [00:00<00:00, 23087.91it/s]
/root/Projects/Kaggle/linking-writing/m4_feats_functions.py:360: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
/root/Projects/Kaggle/linking-writing/m4_feats_functions.py:361: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
/root/Projects/Kaggle/linking-writing/m4_feats_functions.py:362: PerformanceWarning: DataFrame is highly fragment

Engineering input words data
Engineering ratios data


100%|██████████| 3/3 [00:00<00:00, 2775.23it/s]


(2471, 375)

In [9]:
train_feats = train_feats.merge(train_, on='id', how='left')
test_feats = test_feats.merge(test_, on='id', how='left')
train_feats.shape

(2471, 495)

In [10]:
train_feats.to_pickle(f'{FEAT_STORE_DIR}/train_feats_2_1.pkl')
test_feats.to_pickle(f'{FEAT_STORE_DIR}/test_feats_2_1.pkl')

In [5]:
file_name = 'essay_words_feats'

train_essay = getEssays(train_logs)
test_essay = getEssays(test_logs)
train_ = create_word_length_features(train_essay, 'essay_words', 'id', 'words')
test_ = create_word_length_features(test_essay, 'essay_words', 'id', 'words')

train_, test_ = process_feats_action_time_gap(train_logs, test_logs)
train_, test_ = process_feats_time_gap_activity(train_logs, test_logs)


100%|██████████| 3/3 [00:00<00:00, 3413.70it/s]


In [ ]:
file_name = 'base_feats'

preprocessor = Preprocessor(seed=42)
train_feats = preprocessor.make_feats(train_logs)
test_feats = preprocessor.make_feats(test_logs)
nan_cols = train_feats.columns[train_feats.isna().any()].tolist()
train_feats = train_feats.drop(columns=nan_cols)
test_feats = test_feats.drop(columns=nan_cols)

train_feats.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')
test_feats.to_pickle(f'{FEAT_STORE_DIR}/test_{file_name}.pkl')

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 33/33 [01:24<00:00,  2.57s/it, column=word_count_change100, method=kurt]         


Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 14360.23it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering event counts data


...

Engineering text change counts data


100%|██████████| 3/3 [00:00<00:00, 31855.47it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering punctuation counts data


100%|██████████| 3/3 [00:00<00:00, 38362.54it/s]
/root/Projects/Kaggle/linking-writing/m4_feats_functions.py:360: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
/root/Projects/Kaggle/linking-writing/m4_feats_functions.py:361: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
/root/Projects/Kaggle/linking-writing/m4_feats_functions.py:362: PerformanceWarning: DataFrame is highly fragment

Engineering input words data
Engineering ratios data


In [8]:
train_essay_words = create_word_length_features(train_essay, 'essay_words', 'id', 'words')
test_essay_words = create_word_length_features(test_essay, 'essay_words', 'id', 'words')
train_essay_words.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')
test_essay_words.to_pickle(f'{FEAT_STORE_DIR}/test_{file_name}.pkl')

In [8]:
file_name = 'at_by_bucket'
train_action_buckets, test_action_buckets = action_time_by_bucket_feats(train_logs, test_logs)
train_action_buckets.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')
test_action_buckets.to_pickle(f'{FEAT_STORE_DIR}/test_{file_name}.pkl')

In [9]:
file_name = 'at_by_activ'
train_at_by_act, test_at_by_act = process_action_time_activity(train_logs, test_logs)
train_at_by_act.to_pickle(f'{FEAT_STORE_DIR}/train_{file_name}.pkl')
test_at_by_act.to_pickle(f'{FEAT_STORE_DIR}/test_{file_name}.pkl')

In [11]:
file_name = 'adj_eff_time'
train_adj_eff_time, test_adj_eff_time = process_adjusted_eff_time(train_logs, test_logs)
train_adj_eff_time.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_adj_eff_time.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

In [13]:
file_name = 'rep_cut'
train_rep_cut, test_rep_cut = process_re_cut_essays(train_logs, test_logs)
train_rep_cut.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_rep_cut.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

100%|██████████| 2470/2470 [00:00<00:00, 4126.84it/s]


In [15]:
file_name = 'action_time_gap'
train_at_gap, test_at_gap = process_feats_action_time_gap(train_logs, test_logs)
train_at_gap.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_at_gap.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

In [17]:
file_name = 'action_time_gap_by_acti'
train_feats, test_feats = process_feats_time_gap_activity(train_logs, test_logs)
train_feats.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_feats.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

In [3]:
file_name = 'IKI'
train_IKI = train_logs.groupby(['id']).apply(calculate_pause_features).reset_index()
test_IKI = test_logs.groupby(['id']).apply(calculate_pause_features).reset_index()
train_IKI.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_IKI.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

In [3]:
file_name = 'train_wc_chage'
train_feats = create_feats_wc_change(train_logs)
test_feats = create_feats_wc_change(test_logs)
train_feats.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_feats.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

In [4]:
file_name = 'wpm_feats'
train_feats, test_feats = wpm_feats(train_logs, test_logs)
train_feats.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_feats.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

In [7]:
file_name = 'essay_paste_words'
train_feats, test_feats = essay_paste_words(train_logs, test_logs)
train_feats.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_feats.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

100%|██████████| 317/317 [00:00<00:00, 16520.80it/s]


In [6]:
train_essays = getEssays(train_logs)
test_essays = getEssays(test_logs)

# Sentence features for train dataset
train_essays = getEssays(train_logs)
train_sent_df = split_essays_into_sentences(train_essays)
train_sent_agg_df = compute_sentence_aggregations(train_sent_df)

# Paragraph features for train dataset
train_paragraph_df = split_essays_into_paragraphs(train_essays)
train_paragraph_agg_df = compute_paragraph_aggregations(train_paragraph_df)

# Features for test dataset
test_sent_agg_df = compute_sentence_aggregations(split_essays_into_sentences(test_essays))
test_paragraph_agg_df = compute_paragraph_aggregations(split_essays_into_paragraphs(test_essays))

file_name = 'essay_sen'
train_sent_agg_df.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_sent_agg_df.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

file_name = 'essay_par'
train_paragraph_agg_df.to_pickle(f'{FEAT_STORE_DIR}/train/train_{file_name}.pkl')
test_paragraph_agg_df.to_pickle(f'{FEAT_STORE_DIR}/test/test_{file_name}.pkl')

100%|██████████| 2471/2471 [00:04<00:00, 613.59it/s]
